# ML Pipeline Preparation
Last updated: 15/08/2022

This notebook builds a machine learning pipeline that extracts text features from the messages and categories dataset, trains a multi-label classifier, tunes its hyperparameters using grid search, outputs the model performance and exports the optimum model in a pickle file.

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import pickle

import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages_categories', con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
Y = df.iloc[:,4:]

In [4]:
# print(X.head())
print(Y.head())

   related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone      ...        \
0                  0         0         0            0      ...         
1                  0         0         0            0      ...         
2                  0         0         0            0      ...         
3                  0         0         0            0      ...         
4                  0         0         0            0      ...         

   aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
0            0                     0  

In [5]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [6]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
detected_urls = re.findall(url_regex, str(df['message'].values))
len(detected_urls)
# URL remover not needed

0

In [7]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline1 = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('moc', MultiOutputClassifier(RandomForestClassifier()))
            ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [10]:
pipeline1.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
# predict on test data
y_pred1 = pipeline1.predict(X_test)

In [12]:
print("Results with baseline Random Forest Classifier:")
print(classification_report(y_test, y_pred1, target_names=Y.columns))

Results with baseline Random Forest Classifier:
                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      4999
               request       0.85      0.37      0.52      1100
                 offer       0.00      0.00      0.00        38
           aid_related       0.75      0.52      0.61      2707
          medical_help       0.62      0.12      0.21       502
      medical_products       0.62      0.08      0.14       318
     search_and_rescue       0.44      0.04      0.07       175
              security       0.00      0.00      0.00       129
              military       0.66      0.09      0.15       220
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.35      0.50       436
                  food       0.85      0.40      0.54       728
               shelter       0.81      0.28      0.41       568
              clothing       0.58      0.14      0.22  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
#pipeline1.get_params()

In [14]:
parameters = {
    'vect__ngram_range' : ((1, 1), (1, 2)),
    'tfidf__smooth_idf' : [True, False],
    'moc__estimator__max_depth' : [40, 50]
}

cv1 = GridSearchCV(pipeline1, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
from time import gmtime, strftime

In [16]:
print("Pipeline 1 started: ", strftime("%Y-%m-%d %H:%M:%S", gmtime()))
cv1.fit(X_train, y_train)
y_pred2 = cv1.predict(X_test)

print("Best Parameters for Random Forest Classifier:",cv1.best_params_)
print("Results with tuned Random Forest Classifier:")
print(classification_report(y_test, y_pred2, target_names=Y.columns))
print("Pipeline 1 completed successfully: ", strftime("%Y-%m-%d %H:%M:%S", gmtime()))
# the tuned hyperparameters slightly improved average precision but decreased average recall 

Pipeline 1 started:  2022-08-15 21:52:37
Best Parameters for Random Forest Classifier: {'moc__estimator__max_depth': 40, 'tfidf__smooth_idf': True, 'vect__ngram_range': (1, 2)}
Results with tuned Random Forest Classifier:
                        precision    recall  f1-score   support

               related       0.77      0.99      0.87      4999
               request       0.92      0.17      0.29      1100
                 offer       0.00      0.00      0.00        38
           aid_related       0.78      0.34      0.47      2707
          medical_help       0.90      0.02      0.04       502
      medical_products       0.67      0.01      0.01       318
     search_and_rescue       0.80      0.02      0.04       175
              security       0.00      0.00      0.00       129
              military       1.00      0.00      0.01       220
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.00      0.00       436
         

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
class StopwordCounter(BaseEstimator, TransformerMixin):

    def count_stopwords(self, text):
        clean_tokens = tokenize(text)
        stop_words = set(stopwords.words('english'))  
        stopwords_x = [w for w in clean_tokens if w in stop_words]
        return len(stopwords_x)

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_counted = pd.Series(X).apply(self.count_stopwords)
        return pd.DataFrame(X_counted)

In [20]:
pipeline2 = Pipeline([
                ('features', FeatureUnion([
                    ('text_pipeline', Pipeline([
                        ('vect', CountVectorizer(tokenizer=tokenize)),
                        ('tfidf', TfidfTransformer())
                    ])),
                    
                    ('stopword_count', StopwordCounter())
                ])),
    
                ('moc', MultiOutputClassifier(DecisionTreeClassifier(max_depth=40)))
            ])

In [17]:
#pipeline2.get_params()

In [23]:
parameters2 = {
    'features__text_pipeline__vect__ngram_range' : ((1, 1), (1, 2)),
    'features__text_pipeline__tfidf__smooth_idf' : [True, False],
    'moc__estimator__min_samples_leaf' : [1, 2, 4]
}

In [ ]:
print("Pipeline 2 started: ", strftime("%Y-%m-%d %H:%M:%S", gmtime()))
cv2 = GridSearchCV(pipeline2, param_grid=parameters2)
cv2.fit(X_train, y_train)
y_pred3 = cv2.predict(X_test)

print("Best Parameters for Decision Tree Classifier:",cv2.best_params_)
print("Results with tuned Decision Tree Classifier:")
print(classification_report(y_test, y_pred3, target_names=Y.columns))
print("Pipeline 2 completed successfully: ", strftime("%Y-%m-%d %H:%M:%S", gmtime()))
# adding the stopwords count feature and using decision tree classifier led to a lower average precision 
# but higher average recall and slightly lower F1 score

Pipeline 2 started:  2022-08-15 22:57:39


We proceed with the original random forest classifier which has a higher average precision.

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv1, open('classifier.pkl','wb'))

### 10. The machine learning pipeline in this notebook is productionised in `train_classifier.py`